In [389]:
import numpy as np
from scipy import io, integrate, linalg, signal
from scipy.sparse.linalg import cg, eigs

# Capítulo 5

## Exemplo 5.1

In [390]:
A = np.array([[-7, 0, 2], [7, -8, 0], [0, 8, -8]])
B = np.array([-50, -1, 0])
C = linalg.solve(A, B)
print(C)

[9.57142857 8.5        8.5       ]


## Exemplo 5.2

In [391]:
A = np.array([[-250, 0, 40], [240, -250, 0], [0, 240, -250]])
B = np.array([-6500, -2500, -2500])
# T = linalg.solve(A, B)
T = linalg.inv(A) @ B
print(T)

[34.17536221 42.80834772 51.09601381]


## Newton-Rhapson para Sistemas de Equações Não Lineares

In [392]:
def newton_rhapson(f, df, x_est, i_max=20):
    x = [list(x_est)]
    for i in range(i_max):
        X = np.array(x[i])
        dF = np.array(df(X))
        F = np.array(f(X))
        X = X - linalg.inv(dF) @ F
        x.append(list(X))
    return x

## Exemplo da Seção 5.2.1

In [393]:
def f(x):
    Ca = x[0]
    Cb = x[1]
    u = Ca + 0.06*Ca*Cb - 200
    v = Cb + 0.06*Ca*Cb - 200
    return [u, v]

def df(x):
    Ca = x[0]
    Cb = x[1]
    du_dCa = 1 + 0.06*Cb
    du_dCb = 0.06 * Ca
    dv_dCa = 0.06 * Cb
    dv_dCb = 1 + 0.06*Ca
    return [[du_dCa, du_dCb], [dv_dCa, dv_dCb]]

In [394]:
C = newton_rhapson(f, df, [200, 200], 6)
for i, v in enumerate(C):
    print(i, v)

0 [200, 200]
1 [103.99999999999977, 104.0]
2 [62.979228486646804, 62.97922848664683]
3 [51.18114093652843, 51.18114093652841]
4 [50.011720627032815, 50.01172062703282]
5 [50.00000117724716, 50.000001177247164]
6 [50.000000000000014, 50.00000000000002]


### Problema Proposto 5.1

In [395]:
A = np.array([[0.50, 0.33, 0], [0.50, 0.33, 0.70], [0, 0.34, 0.30]])
print(A)

[[0.5  0.33 0.  ]
 [0.5  0.33 0.7 ]
 [0.   0.34 0.3 ]]


In [396]:
B = np.array([0.30, 0.40, 0.30]) * 100
print(B)

[30. 40. 30.]


In [397]:
F = linalg.inv(A) @ B
print(F)
print(sum(F))

[10.08403361 75.6302521  14.28571429]
100.0


### Problema Proposto 5.2

In [398]:
Qin, Q32, Q43 = 10, 5, 3
V1, V2, V3, V4 = 25, 75, 100, 25
k1, k2, k3, k4 = 0.075, 0.15, 0.4, 0.1
Cain = 1

In [399]:
A = np.array([
    [-1, 0, 0, 0],
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1]
])
print(A)

[[-1  0  0  0]
 [ 1 -1  0  0]
 [ 0  1 -1  0]
 [ 0  0  1 -1]]


In [400]:
B = np.array([-Qin, -Q32, -Q43+Q32, Q43])
print(B)

[-10  -5   2   3]


In [401]:
Q = linalg.inv(A) @ B
print(Q)
Q12, Q23, Q34, Qout = Q

[10. 15. 13. 10.]


In [402]:
A = np.array([
    [-Q12-k1*V1, 0, 0, 0],
    [Q12, -Q23-k2*V2, Q32, 0],
    [0, Q23, -Q34-Q32-k3*V3, Q43],
    [0, 0, Q34, -Qout-Q43-k4*V4]
])
print(A)

[[-11.875   0.      0.      0.   ]
 [ 10.    -26.25    5.      0.   ]
 [  0.     15.    -58.      3.   ]
 [  0.      0.     13.    -15.5  ]]


In [403]:
B = np.array([-Qin*Cain, 0, 0, 0])
print(B)

[-10   0   0   0]


In [404]:
C = linalg.inv(A) @ B
print(C)

[0.84210526 0.33821858 0.091437   0.0766891 ]


### Problema Proposto 5.3

In [405]:
def f(x):
    Ca, Cb, Cc, Cd = x
    Cain, Cbin, Ccin, Cdin = 0.7, 0.4, 0, 0
    F = 5 / 60
    k = 0.855
    V = 40
    ua = F*(Cain - Ca) - k*Ca*Cb*V
    ub = F*(Cbin - Cb) - k*Ca*Cb*V
    uc = F*(Ccin - Cc) + k*Ca*Cb*V
    ud = F*(Cdin - Cd) + k*Ca*Cb*V
    return [ua, ub, uc, ud]

In [406]:
def df(x):
    Ca, Cb, Cc, Cd = x
    # Cain, Cbin, Ccin, Cdin = 0.7, 0.4, 0, 0
    F = 5 / 60
    k = 0.855
    V = 40
    dua_dCa = -F - k*Cb*V
    dua_dCb = -k*Ca*V
    dua_dCc = 0
    dua_dCd = 0
    dub_dCa = -k*Cb*V
    dub_dCb = -F - k*Ca*V
    dub_dCc = 0
    dub_dCd = 0
    duc_dCa = k*Cb*V
    duc_dCb = k*Ca*V
    duc_dCc = -F
    duc_dCd = 0
    dud_dCa = k*Cb*V
    dud_dCb = k*Ca*V
    dud_dCc = 0
    dud_dCd = -F
    return [
        [dua_dCa, dua_dCb, dua_dCc, dua_dCd],
        [dub_dCa, dub_dCb, dub_dCc, dub_dCd],
        [duc_dCa, duc_dCb, duc_dCc, duc_dCd],
        [dud_dCa, dud_dCb, dud_dCc, dud_dCd]
    ]

In [407]:
C = newton_rhapson(f, df, [0.7, 0.4, 0, 0], 10)
for i, v in enumerate(C):
    print(i, v)
print(sum(C[0]) - sum(C[-1]))

0 [0.7, 0.4, 0, 0]
1 [0.4460171514454998, 0.14601715144549277, 0.25398284855450015, 0.25398284855450015]
2 [0.33750505787243834, 0.037505057872440184, 0.36249494212755806, 0.36249494212755806]
3 [0.30630893802611747, 0.006308938026117705, 0.3936910619738825, 0.3936910619738826]
4 [0.303219955574757, 0.003219955574757077, 0.39678004442524295, 0.396780044425243]
5 [0.30318906358140985, 0.0031890635814098996, 0.3968109364185901, 0.39681093641859017]
6 [0.30318906049115857, 0.0031890604911586147, 0.3968109395088414, 0.3968109395088414]
7 [0.3031890604911585, 0.003189060491158584, 0.39681093950884144, 0.39681093950884144]
8 [0.3031890604911585, 0.003189060491158584, 0.39681093950884144, 0.39681093950884144]
9 [0.3031890604911585, 0.003189060491158584, 0.39681093950884144, 0.39681093950884144]
10 [0.3031890604911585, 0.003189060491158584, 0.39681093950884144, 0.39681093950884144]
0.0
